#### Light GBMでモデルを構築
LB:348.50341
 - description translatedをtf-idf
 - optunaでハイパラチューニング

In [1]:
import os
os.chdir('/Users/kinoshitashouhei/Desktop/competitions/05_Prob_Space/Kiva/')

In [2]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc
import lightgbm as lgb
import optuna.integration.lightgbm as lgb_optuna
import pickle
import optuna
import warnings

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split
from sklearn.pipeline import Pipeline

# 自作関数のインポート
from functions.common import *
from functions import preprocessing

warnings.filterwarnings('ignore')

In [3]:
# 各種設定
N_COMP = 10
DICT_DTYPE = {'LOAN_ID': 'str', 'IMAGE_ID': 'str'}
LIST_COL_DROPS = [COL_LOAN_ID,
                  COL_DESCRIPTION,
                  COL_DESCRIPTION_TRANSLATED,
                  COL_ACTIVITY_NAME,
                  COL_LOAN_USE,
                  COL_IMAGE_ID,
                  COL_TOWN_NAME,
                  COL_COUNTRY_NAME,
                  COL_TAGS]

LIST_TARGET_ENC = [COL_ORIGINAL_LANGUAGE]
LIST_LABEL_ENC = [COL_SECTOR_NAME, COL_CURRENCY_POLICY, COL_CURRENCY, COL_REPAYMENT_INTERVAL, COL_DISTRIBUTION_MODEL]
LIST_LDA_DESCRIPTINO_TRANSLATED = [f'COL_LDA_DESCRIPTION_TRANSLATED_{i+1}' for i in range(N_COMP)]
LIST_LDA_LOAN_USE = [f'COL_LDA_LOAN_USE_{i+1}' for i in range(N_COMP)]

In [5]:
# csvデータの読み込み
df_train = pd.read_csv('data/train.csv', dtype=DICT_DTYPE)
df_test = pd.read_csv('data/test.csv', dtype=DICT_DTYPE)

In [6]:
# 欠損値の補完
df_train = preprocessing.fill_na_DESCRIPTION_TRANSLATED(df_train)

# テキストを前処理(DESCRIPTION_TRANSLATED)
df_train[COL_DESCRIPTION_TRANSLATED] = df_train[COL_DESCRIPTION_TRANSLATED].apply(preprocessing.replace_str)
df_train[COL_DESCRIPTION_TRANSLATED] = df_train[COL_DESCRIPTION_TRANSLATED].apply(preprocessing.lower_text)
df_test[COL_DESCRIPTION_TRANSLATED] = df_test[COL_DESCRIPTION_TRANSLATED].apply(preprocessing.replace_str)
df_test[COL_DESCRIPTION_TRANSLATED] = df_test[COL_DESCRIPTION_TRANSLATED].apply(preprocessing.lower_text)

In [14]:
# TfidfVectorizer
vec_tfidf = TfidfVectorizer()

# ベクトル化
vec_tfidf.fit(df_train[COL_DESCRIPTION_TRANSLATED])
X = vec_tfidf.transform(df_train[COL_DESCRIPTION_TRANSLATED])
X_inference = vec_tfidf.transform(df_test[COL_DESCRIPTION_TRANSLATED])

print('Vocabulary size: {}'.format(len(vec_tfidf.vocabulary_)))

Vocabulary size: 53160


In [8]:
# 学習に使用するデータを設定
y = df_train[COL_LOAN_AMOUNT]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train) 

In [9]:
# ベストなパラメータ、途中経過を保存する
params = {
    'objective': 'regression',
    'metric': 'mae',
    "verbosity": -1,
    "boosting_type": "gbdt",
}

best_params, history = {}, []

# LightGBM学習
gbm = lgb_optuna.train(params,
                lgb_train,
                num_boost_round=200,
                valid_sets=[lgb_train, lgb_eval],
                early_stopping_rounds=50
               )

best_params = gbm.params

[I 2021-12-27 18:36:52,998] A new study created in memory with name: no-name-123bdc33-f58c-4211-a9f9-83253eb2b1eb
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	valid_0's l1: 536.689	valid_1's l1: 533.2
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 511.328	valid_1's l1: 508.498
[3]	valid_0's l1: 487.299	valid_1's l1: 485.044
[4]	valid_0's l1: 467.44	valid_1's l1: 466.025
[5]	valid_0's l1: 449.869	valid_1's l1: 449.304
[6]	valid_0's l1: 433.984	valid_1's l1: 433.961
[7]	valid_0's l1: 421.155	valid_1's l1: 421.757
[8]	valid_0's l1: 410.34	valid_1's l1: 411.599
[9]	valid_0's l1: 400.162	valid_1's l1: 402.085
[10]	valid_0's l1: 391.569	valid_1's l1: 393.823
[11]	valid_0's l1: 384.085	valid_1's l1: 386.864
[12]	valid_0's l1: 376.694	valid_1's l1: 379.787
[13]	valid_0's l1: 370.182	valid_1's l1: 373.642
[14]	valid_0's l1: 364.542	valid_1's l1: 368.307
[15]	valid_0's l1: 359.921	valid_1's l1: 364.137
[16]	valid_0's l1: 355.211	valid_1's l1: 360.118
[17]	valid_0's l1: 351.67	valid_1's l1: 356.924
[18]	valid_0's l1: 348.433	valid_1's l1: 354.053
[19]	valid_0's l1: 345.429	valid_1's l1: 351.264
[20]	valid_0's l1: 342

feature_fraction, val_score: 303.624967:  14%|#4        | 1/7 [00:25<02:30, 25.16s/it]

[1]	valid_0's l1: 538.839	valid_1's l1: 535.487
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 516.526	valid_1's l1: 514.044
[3]	valid_0's l1: 493.988	valid_1's l1: 492.025
[4]	valid_0's l1: 474.617	valid_1's l1: 473.037
[5]	valid_0's l1: 456.495	valid_1's l1: 455.735
[6]	valid_0's l1: 440.26	valid_1's l1: 440.163
[7]	valid_0's l1: 428.157	valid_1's l1: 428.901
[8]	valid_0's l1: 415.571	valid_1's l1: 416.832
[9]	valid_0's l1: 405.912	valid_1's l1: 407.731
[10]	valid_0's l1: 397.268	valid_1's l1: 399.217
[11]	valid_0's l1: 389.008	valid_1's l1: 391.272
[12]	valid_0's l1: 382.642	valid_1's l1: 385.354
[13]	valid_0's l1: 375.505	valid_1's l1: 378.559
[14]	valid_0's l1: 369.622	valid_1's l1: 373.068
[15]	valid_0's l1: 364.03	valid_1's l1: 367.899
[16]	valid_0's l1: 359.92	valid_1's l1: 364.497
[17]	valid_0's l1: 354.844	valid_1's l1: 359.814
[18]	valid_0's l1: 350.62	valid_1's l1: 356.007
[19]	valid_0's l1: 346.643	valid_1's l1: 352.441
[20]	valid_0's l1: 34

feature_fraction, val_score: 303.624967:  29%|##8       | 2/7 [01:05<02:50, 34.07s/it]

...

min_data_in_leaf, val_score: 290.720067:  60%|######    | 3/5 [02:46<01:56, 58.05s/it]

[1]	valid_0's l1: 529.944	valid_1's l1: 527.88
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 499.066	valid_1's l1: 498.833
[3]	valid_0's l1: 470.571	valid_1's l1: 471.961
[4]	valid_0's l1: 446.132	valid_1's l1: 449.749
[5]	valid_0's l1: 424.832	valid_1's l1: 430.48
[6]	valid_0's l1: 405.726	valid_1's l1: 413.618
[7]	valid_0's l1: 388.78	valid_1's l1: 398.755
[8]	valid_0's l1: 373.863	valid_1's l1: 386.122
[9]	valid_0's l1: 360.493	valid_1's l1: 375.044
[10]	valid_0's l1: 349.285	valid_1's l1: 365.958
[11]	valid_0's l1: 338.92	valid_1's l1: 357.659
[12]	valid_0's l1: 329.893	valid_1's l1: 350.474
[13]	valid_0's l1: 321.789	valid_1's l1: 344.493
[14]	valid_0's l1: 314.721	valid_1's l1: 339.256
[15]	valid_0's l1: 308.286	valid_1's l1: 334.771
[16]	valid_0's l1: 302.705	valid_1's l1: 331.069
[17]	valid_0's l1: 297.7	valid_1's l1: 327.864
[18]	valid_0's l1: 293.059	valid_1's l1: 324.933
[19]	valid_0's l1: 288.736	valid_1's l1: 322.331
[20]	valid_0's l1: 284.

min_data_in_leaf, val_score: 290.720067:  80%|########  | 4/5 [03:53<01:01, 61.53s/it]

[1]	valid_0's l1: 529.8	valid_1's l1: 527.703
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 498.824	valid_1's l1: 498.645
[3]	valid_0's l1: 470.023	valid_1's l1: 471.28
[4]	valid_0's l1: 445.601	valid_1's l1: 448.849
[5]	valid_0's l1: 424.278	valid_1's l1: 429.219
[6]	valid_0's l1: 405.647	valid_1's l1: 412.252
[7]	valid_0's l1: 389.013	valid_1's l1: 397.361
[8]	valid_0's l1: 374.081	valid_1's l1: 384.392
[9]	valid_0's l1: 361.018	valid_1's l1: 372.962
[10]	valid_0's l1: 350.142	valid_1's l1: 363.443
[11]	valid_0's l1: 340.001	valid_1's l1: 355.142
[12]	valid_0's l1: 331.39	valid_1's l1: 348.237
[13]	valid_0's l1: 323.427	valid_1's l1: 341.879
[14]	valid_0's l1: 316.382	valid_1's l1: 336.624
[15]	valid_0's l1: 310.142	valid_1's l1: 331.976
[16]	valid_0's l1: 304.639	valid_1's l1: 328.172
[17]	valid_0's l1: 299.894	valid_1's l1: 325.022
[18]	valid_0's l1: 295.458	valid_1's l1: 322.102
[19]	valid_0's l1: 291.261	valid_1's l1: 319.294
[20]	valid_0's l1: 28

min_data_in_leaf, val_score: 290.720067: 100%|##########| 5/5 [04:56<00:00, 59.32s/it]


In [10]:
best_params

{'objective': 'regression',
 'metric': 'l1',
 'verbosity': -1,
 'boosting_type': 'gbdt',
 'feature_pre_filter': False,
 'lambda_l1': 0.00031094203181686894,
 'lambda_l2': 7.90106551315504e-07,
 'num_leaves': 184,
 'feature_fraction': 0.7,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 5,
 'num_iterations': 200,
 'early_stopping_round': 50}

In [11]:
params = {'objective': 'regression',
          'metric': 'l1',
          'verbosity': -1,
          'boosting_type': 'gbdt',
          'feature_pre_filter': False,
          'lambda_l1': 0.00031094203181686894,
          'lambda_l2': 7.90106551315504e-07,
          'num_leaves': 184,
          'feature_fraction': 0.7,
          'bagging_fraction': 1.0,
          'bagging_freq': 0,
          'min_child_samples': 5,
          'num_iterations': 200,
          'early_stopping_round': 50}

In [12]:
# モデルの学習
model = lgb.train(params,
                  train_set=lgb_train, # トレーニングデータの指定
                  valid_sets=lgb_eval, # 検証データの指定
                  )

[1]	valid_0's l1: 528.055
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 499.005
[3]	valid_0's l1: 472.501
[4]	valid_0's l1: 450.321
[5]	valid_0's l1: 431.296
[6]	valid_0's l1: 414.532
[7]	valid_0's l1: 399.822
[8]	valid_0's l1: 387.441
[9]	valid_0's l1: 376.754
[10]	valid_0's l1: 367.535
[11]	valid_0's l1: 359.3
[12]	valid_0's l1: 352.408
[13]	valid_0's l1: 346.694
[14]	valid_0's l1: 341.294
[15]	valid_0's l1: 336.931
[16]	valid_0's l1: 333.022
[17]	valid_0's l1: 329.788
[18]	valid_0's l1: 326.997
[19]	valid_0's l1: 324.214
[20]	valid_0's l1: 322.15
[21]	valid_0's l1: 319.797
[22]	valid_0's l1: 317.854
[23]	valid_0's l1: 315.736
[24]	valid_0's l1: 314.199
[25]	valid_0's l1: 312.526
[26]	valid_0's l1: 311.065
[27]	valid_0's l1: 310.194
[28]	valid_0's l1: 308.92
[29]	valid_0's l1: 308.086
[30]	valid_0's l1: 307.158
[31]	valid_0's l1: 306.425
[32]	valid_0's l1: 305.664
[33]	valid_0's l1: 305.052
[34]	valid_0's l1: 304.438
[35]	valid_0's l1: 303.675
[36]	va

In [17]:
sample_sub = pd.read_csv('data/sample_submission.csv')
preds = model.predict(X_inference)
sample_sub[COL_LOAN_AMOUNT] = preds

sample_sub.to_csv('submit/sample_sub_tfidf.csv', index=None)

#### モデルの保存

In [20]:
MODEL_NAME = 'models/lightgbm_tfidf_description_translated.pkl'
pickle.dump(model, open(MODEL_NAME, 'wb'))